In [1]:
import pandas as pd
# Set option to display all columns
pd.set_option('display.max_columns', None)


In [2]:
data = pd.read_parquet("outputs_new/dedupe_data_out.parquet")
data

,record_id,server_name,backend,doi,doi_url,landing_page_url,title,authors_flat,institutions_flat,countries_flat,relations_json,version_label,is_version_of,is_preprint_of,has_preprint,has_review,has_published_version,published_version_ids_json,version_of_ids_json,update_to_json,raw_relationships_json,records_hierarchy,date_first_seen,publication_year_first_seen,records_hierarchy_backup,parent_record_id,dup_group_id,authors_fp_tokenbag,title_clean_v2,authors_fp_last_initial,authors_fp_last
0,crossref::10.21467/preprints.48,AIJR Preprints,crossref,10.21467/preprints.48,https://doi.org/10.21467/preprints.48,https://preprints.aijr.org/index.php/ap/prepri...,"Bird’s Eye View on the Diagnosis, Treatment, &...","Panchalingala, Sai Bhargavi",None,None,None,None,,,,,false,None,None,None,None,parent,2020-05-03,2020,parent,None,None,None,None,None,None
1,crossref::10.21467/preprints.43,AIJR Preprints,crossref,10.21467/preprints.43,https://doi.org/10.21467/preprints.43,https://preprints.aijr.org/index.php/ap/prepri...,Doxycycline and Minocycline Drugs as a Treatme...,"Mostafa, Mohamed",None,None,None,None,,,,,false,None,None,None,None,parent,2020-04-25,2020,parent,None,None,mohamed_mostafa,doxycycline and minocycline drugs as a treatme...,mostafa|m,None
2,crossref::10.21467/preprints.39,AIJR Preprints,crossref,10.21467/preprints.39,https://doi.org/10.21467/preprints.39,https://preprints.aijr.org/index.php/ap/prepri...,A Genetic Perspective of 2019-nCoV in Relation...,"Dasgupta, Rimjhim",None,None,None,None,,,,,false,None,None,None,None,parent,2020-04-16,2020,parent,None,None,dasgupta_rimjhim,a genetic perspective of 2019 ncov in relation...,dasgupta|r,None
3,crossref::10.21467/preprints.38,AIJR Preprints,crossref,10.21467/preprints.38,https://doi.org/10.21467/preprints.38,https://preprints.aijr.org/index.php/ap/prepri...,Marine Algae as a Natural Source for Antiviral...,"Musale, Amar S; G., Raja Krishna Kumar; Sapre,...",None,None,None,None,,,,,false,None,None,None,None,parent,2020-04-15,2020,parent,None,None,None,None,None,None
4,crossref::10.21467/preprints.36,AIJR Preprints,crossref,10.21467/preprints.36,https://doi.org/10.21467/preprints.36,https://preprints.aijr.org/index.php/ap/prepri...,Possible Prevention of COVID 19 by Using Linol...,"Subhash, Venkata; G, Raja Krishna Kumar; Sapre...",None,None,None,None,,,,,false,None,None,None,None,parent,2020-04-15,2020,parent,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8410089,openalex::W999325625,viXra,openalex,None,None,https://vixra.org/pdf/1409.0090v1.pdf,Three Objections to Modern Physics,Lubomir Vlcek,None,None,None,None,None,None,None,None,None,None,None,None,None,parent,2014-09-01,2014,parent,None,None,lubomir_vlcek,three objections to modern physics,vlcek|l,None
8410090,openalex::W999460032,viXra,openalex,None,None,https://vixra.org/abs/1112.0094,Particle Mass Ratios,DT Froedge,None,None,None,None,None,None,None,None,None,None,None,None,None,parent,2011-12-01,2011,parent,None,None,dt_froedge,particle mass ratios,froedge|d,None
8410091,openalex::W99967155,viXra,openalex,None,None,https://vixra.org/pdf/1406.0019v1.pdf,Quantum FFF Theory Proposals for Some Unsolved...,Leo Vuyk,None,None,None,None,None,None,None,None,None,None,None,None,None,parent,2014-06-01,2014,parent,None,None,leo_vuyk,quantum fff theory proposals for some unsolved...,vuyk|l,None
8410092,openalex::W999790414,viXra,openalex,None,None,https://vixra.org/pdf/1306.0105v3.pdf,Investigation of the Formalism of Particle Dyn...,Chi-Yi Chen,None,None,None,None,None,None,None,None,None,None,None,None,None,parent,2013-06-01,2013,parent,None,None,chen_chi_yi,investigation of the formalism of particle dyn...,chen|c,None


In [3]:
data.columns

Index(['record_id', 'server_name', 'backend', 'doi', 'doi_url',
       'landing_page_url', 'title', 'authors_flat', 'institutions_flat',
       'countries_flat', 'relations_json', 'version_label', 'is_version_of',
       'is_preprint_of', 'has_preprint', 'has_review', 'has_published_version',
       'published_version_ids_json', 'version_of_ids_json', 'update_to_json',
       'raw_relationships_json', 'records_hierarchy', 'date_first_seen',
       'publication_year_first_seen', 'records_hierarchy_backup',
       'parent_record_id', 'dup_group_id', 'authors_fp_tokenbag',
       'title_clean_v2', 'authors_fp_last_initial', 'authors_fp_last'],
      dtype='object')

In [4]:
import pandas as pd
import numpy as np

# Columns to duplicate
cols_to_duplicate = ['records_hierarchy', 'parent_record_id','dup_group_id']

# Duplicate the columns with new names
data[['records_hierarchy_dedupe_backup', 'parent_record_id_dedupe_backup', 'dup_group_id_dedupe_backup']] = data[cols_to_duplicate].copy()

data.columns


Index(['record_id', 'server_name', 'backend', 'doi', 'doi_url',
       'landing_page_url', 'title', 'authors_flat', 'institutions_flat',
       'countries_flat', 'relations_json', 'version_label', 'is_version_of',
       'is_preprint_of', 'has_preprint', 'has_review', 'has_published_version',
       'published_version_ids_json', 'version_of_ids_json', 'update_to_json',
       'raw_relationships_json', 'records_hierarchy', 'date_first_seen',
       'publication_year_first_seen', 'records_hierarchy_backup',
       'parent_record_id', 'dup_group_id', 'authors_fp_tokenbag',
       'title_clean_v2', 'authors_fp_last_initial', 'authors_fp_last',
       'records_hierarchy_dedupe_backup', 'parent_record_id_dedupe_backup',
       'dup_group_id_dedupe_backup'],
      dtype='object')

In [5]:
bad_values = {"others", "part_of", "comment", "review"}

mask = ~data["records_hierarchy_backup"].astype(str).str.contains(
    r"\b(" + "|".join(bad_values) + r")\b",
    case=False,
    regex=True
)

data_clean = data[mask].copy()


/tmp/ipykernel_188816/2399585284.py:3: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  mask = ~data["records_hierarchy_backup"].astype(str).str.contains(


In [6]:
data_clean['records_hierarchy'].unique()

array(['parent', 'parent - duplicate (SSRN)',
       'parent - duplicate (Preprints.org)',
       'parent - duplicate (AMRC Open Research)',
       'parent - duplicate (APSA Preprints)',
       'parent - duplicate (medRxiv)',
       'parent - duplicate (ResearchGate)',
       'parent - duplicate (ScienceOpen Preprints)',
       'parent - duplicate (arXiv)', 'parent - duplicate (SocArXiv)',
       'parent - duplicate (Open Science Framework)',
       'parent - duplicate (PREPRINTS.RU)',
       'parent - duplicate (MediArXiv)',
       'parent - duplicate (ARPHA Preprints)',
       'parent - duplicate (Research Square)',
       'parent - duplicate (Authorea Inc.)',
       'parent - duplicate (bioRxiv)', 'parent - duplicate (ART-Dok)',
       'parent - duplicate (HAL)',
       'parent - duplicate (Social Science Open Access Repository)',
       'parent - duplicate (Humanities Commons CORE)',
       'parent - duplicate (Advance)', 'parent - duplicate (PsyArXiv)',
       'parent - duplicate 

In [7]:
data_clean['records_hierarchy_backup'].value_counts().head(60)

records_hierarchy_backup
parent                              7950093
version                              105977
publish_version                        9351
mirror (arXiv)                         8286
mirror (AgEcon Search)                 6702
child                                  2028
mirror (ResearchGate)                  1633
correction                              355
mirror (Zenodo)                         297
mirror (SSRN)                            36
mirror (Open Science Framework)          31
mirror (bioRxiv)                         29
mirror (Humanities Commons CORE)         24
parent_duplicate                          3
mirror (eLife)                            3
mirror (Research Square)                  2
mirror (CERN document server)             2
mirror (TechRxiv)                         1
mirror (EarthArXiv)                       1
mirror (Oroboros Instruments)             1
mirror (medRxiv)                          1
mirror (Thesis Commons)                   1
mirror 

In [8]:
import time
import pandas as pd
import numpy as np
from typing import Dict, Any, Optional

# ------------------------------------------------------------
# Union-Find (Disjoint Set Union) for fast connected-components
# ------------------------------------------------------------
class _UnionFind:
    __slots__ = ("parent", "size")

    def __init__(self, n: int):
        self.parent = np.arange(n, dtype=np.int32)
        self.size = np.ones(n, dtype=np.int32)

    def find(self, x: int) -> int:
        parent = self.parent
        while parent[x] != x:
            parent[x] = parent[parent[x]]
            x = parent[x]
        return x

    def union(self, a: int, b: int) -> bool:
        pa = self.find(a)
        pb = self.find(b)
        if pa == pb:
            return False
        # union by size
        if self.size[pa] < self.size[pb]:
            pa, pb = pb, pa
        self.parent[pb] = pa
        self.size[pa] += self.size[pb]
        return True


def unify_groups_by_version_links(
    df: pd.DataFrame,
    *,
    # core cols
    record_id_col: str = "record_id",
    doi_col: str = "doi",
    is_version_of_col: str = "is_version_of",

    # output cols
    hierarchy_col: str = "records_hierarchy",
    parent_id_col: str = "parent_record_id",
    group_id_col: str = "dup_group_id",

    # parent choice
    date_col: Optional[str] = "date_first_seen",      # if None, will use record_id numeric fallback only
    choose_parent: str = "oldest",                    # "oldest" | "newest"

    # how safe to overwrite
    # - "unlabeled_only": only rows with NA hierarchy are eligible
    # - "parent_only": only rows currently labeled "parent" are eligible
    # - "any": any row is eligible
    overwrite_mode: str = "unlabeled_only",

    # labeling
    label_duplicate_suffix: str = "version-link",     # becomes "parent - duplicate (version-link)"

    # performance
    chunk_size: int = 300_000,                        # only affects the union loop; tune for RAM/CPU
    min_component_size: int = 2,                      # ignore components of size < 2
) -> tuple[pd.DataFrame, Dict[str, Any]]:
    """
    Unify clusters using explicit version relationships (is_version_of) as a HIGH-CONFIDENCE step.

    How it works (fast + robust):
      1) For rows that have either a DOI or an is_version_of list, build a graph where:
         - each row is a node
         - edges exist when two rows share any DOI in their version-set (self DOI + is_version_of DOIs)
      2) Compute connected components via Union-Find (near O(N)).
      3) For each component (size>=2), choose ONE parent (oldest/newest by date, tie-break by record_id numeric).
      4) Update:
         - hierarchy_col: "parent" for parent, "parent - duplicate (version-link)" for others
         - parent_id_col: parent record_id for duplicates
         - group_id_col: stable id "version::<root_record_id>" (can be merged later with other stages)

    Safe overwrite:
      - overwrite_mode controls WHICH rows get updated (eligible), but parents are chosen using all rows in component.

    Returns:
      (df, metrics)
    """

    t0 = time.perf_counter()
    metrics: Dict[str, Any] = {
        "n_rows_df": int(len(df)),
        "n_rows_with_any_version_signal": 0,
        "n_rows_considered_in_graph": 0,
        "n_edges_unions_attempted": 0,
        "n_unions_done": 0,
        "n_components_total": 0,
        "n_components_size_ge_2": 0,
        "n_rows_in_components_size_ge_2": 0,
        "n_rows_eligible_to_update": 0,
        "n_rows_updated": 0,
        "n_parents_set": 0,
        "n_children_set": 0,
        "runtime_s": 0.0,
    }

    # -----------------------------
    # Basic column checks / setup
    # -----------------------------
    for c in (hierarchy_col, parent_id_col, group_id_col):
        if c not in df.columns:
            df[c] = pd.NA

    if record_id_col not in df.columns:
        raise ValueError(f"Missing required column: {record_id_col}")
    if doi_col not in df.columns:
        raise ValueError(f"Missing required column: {doi_col}")
    if is_version_of_col not in df.columns:
        raise ValueError(f"Missing required column: {is_version_of_col}")

    if choose_parent not in {"oldest", "newest"}:
        raise ValueError("choose_parent must be 'oldest' or 'newest'")
    if overwrite_mode not in {"unlabeled_only", "parent_only", "any"}:
        raise ValueError("overwrite_mode must be 'unlabeled_only' | 'parent_only' | 'any'")

    # eligibility mask (safe overwrite)
    h = df[hierarchy_col]
    if overwrite_mode == "any":
        eligible_mask = pd.Series(True, index=df.index)
    elif overwrite_mode == "unlabeled_only":
        eligible_mask = h.isna()
    else:  # parent_only
        eligible_mask = h.astype("string").str.strip().str.lower().eq("parent")

    metrics["n_rows_eligible_to_update"] = int(eligible_mask.sum())

    # ------------------------------------------------------------
    # Build the subset to include in the VERSION graph
    # (We include rows that have DOI OR is_version_of list,
    #  because some rows might have missing doi but have version list.)
    # ------------------------------------------------------------
    doi_s = df[doi_col].astype("string").fillna("").str.strip()
    v_s = df[is_version_of_col].astype("string").fillna("").str.strip()

    has_signal = (doi_s != "") | (v_s != "")
    metrics["n_rows_with_any_version_signal"] = int(has_signal.sum())

    idx = df.index[has_signal].to_numpy()
    if idx.size == 0:
        metrics["runtime_s"] = time.perf_counter() - t0
        return df, metrics

    # Compact indexing for Union-Find arrays
    # local_i: 0..k-1 corresponds to df index idx[local_i]
    k = idx.size
    metrics["n_rows_considered_in_graph"] = int(k)

    # For parent selection: precompute helpers on this subset
    rid = df.loc[idx, record_id_col].astype("string").fillna("")
    # numeric fallback from record_id: extract digits
    rid_num = pd.to_numeric(rid.str.extract(r"(\d+)")[0], errors="coerce")

    if date_col and date_col in df.columns:
        dt = pd.to_datetime(df.loc[idx, date_col], errors="coerce")
    else:
        dt = pd.Series(pd.NaT, index=idx)

    # union-find structure
    uf = _UnionFind(k)

    # ------------------------------------------------------------
    # Union phase: stream through rows and connect by shared DOIs
    # Using a dict: doi -> first_local_index_seen
    # ------------------------------------------------------------
    doi_to_rep: Dict[str, int] = {}

    def _iter_version_set(self_doi: str, is_version_of_str: str):
        """
        Yield DOIs in (self DOI + is_version_of list).
        Designed to be cheap: split ';' only.
        """
        if self_doi:
            yield self_doi
        if is_version_of_str:
            # common case: "doi1;doi2;doi3"
            for part in is_version_of_str.split(";"):
                d = part.strip()
                if d:
                    yield d

    # Process in chunks for cache friendliness (still single pass)
    unions_attempted = 0
    unions_done = 0

    # Pre-pull arrays for speed
    doi_arr = doi_s.loc[idx].to_numpy(dtype=object)
    v_arr = v_s.loc[idx].to_numpy(dtype=object)

    for start in range(0, k, chunk_size):
        end = min(k, start + chunk_size)
        for local_i in range(start, end):
            sd = doi_arr[local_i]
            sv = v_arr[local_i]
            # build edges for this row: union with representative of each doi token
            first_rep = None
            for d in _iter_version_set(sd, sv):
                rep = doi_to_rep.get(d)
                if rep is None:
                    doi_to_rep[d] = local_i
                    if first_rep is None:
                        first_rep = local_i
                else:
                    unions_attempted += 1
                    if uf.union(local_i, rep):
                        unions_done += 1

    metrics["n_edges_unions_attempted"] = int(unions_attempted)
    metrics["n_unions_done"] = int(unions_done)

    # ------------------------------------------------------------
    # Build components: root -> list of local indices
    # ------------------------------------------------------------
    roots = np.empty(k, dtype=np.int32)
    for i in range(k):
        roots[i] = uf.find(i)

    # group by root using pandas (fast enough); avoid Python dict of lists for k large
    comp_df = pd.DataFrame({"root": roots, "local_i": np.arange(k, dtype=np.int32)})
    comp_sizes = comp_df["root"].value_counts()
    metrics["n_components_total"] = int(comp_sizes.shape[0])

    big_roots = comp_sizes[comp_sizes >= min_component_size].index.to_numpy(dtype=np.int32)
    metrics["n_components_size_ge_2"] = int(big_roots.size)
    if big_roots.size == 0:
        metrics["runtime_s"] = time.perf_counter() - t0
        return df, metrics

    # Filter to only components we will act on
    comp_big = comp_df[comp_df["root"].isin(big_roots)].copy()
    metrics["n_rows_in_components_size_ge_2"] = int(len(comp_big))

    # ------------------------------------------------------------
    # Parent selection per component
    # Parent = oldest/newest dt, tie-break by record_id numeric (rid_num)
    # ------------------------------------------------------------
    # Map local_i -> df index
    df_index_for_local = idx  # numpy array length k

    # Create selection frame with sortable keys
    # Note: dt index is df idx; align via local index mapping
    sel_idx = df_index_for_local[comp_big["local_i"].to_numpy()]
    sel = pd.DataFrame({
        "root": comp_big["root"].to_numpy(),
        "df_idx": sel_idx,
        "record_id": df.loc[sel_idx, record_id_col].astype("string").to_numpy(),
        "dt": dt.loc[sel_idx].to_numpy(),
        "rid_num": rid_num.loc[sel_idx].to_numpy(),
    })

    # Sort for parent picking
    asc_dt = True if choose_parent == "oldest" else False
    asc_rn = True if choose_parent == "oldest" else False

    # When dt is NaT, push to end for "oldest" and to start for "newest"
    # pandas sort_values uses na_position; we’ll handle by two sorts:
    na_pos = "last" if choose_parent == "oldest" else "first"

    sel_sorted = sel.sort_values(
        by=["root", "dt", "rid_num"],
        ascending=[True, asc_dt, asc_rn],
        na_position=na_pos,
        kind="mergesort",  # stable
    )

    # parent per root = first row per root
    parents = sel_sorted.groupby("root", sort=False).head(1)[["root", "df_idx", "record_id"]].rename(
        columns={"df_idx": "parent_df_idx", "record_id": "parent_record_id"}
    )

    parent_map_idx = parents.set_index("root")["parent_df_idx"]
    parent_map_rid = parents.set_index("root")["parent_record_id"]

    # ------------------------------------------------------------
    # Apply updates safely (only eligible rows)
    # ------------------------------------------------------------
    # Create per-row root for ALL rows in big components (same as sel_idx rows)
    sel["parent_df_idx"] = sel["root"].map(parent_map_idx).to_numpy()
    sel["parent_record_id"] = sel["root"].map(parent_map_rid).to_numpy()

    # Determine which df rows in these components are eligible to update
    sel["eligible"] = eligible_mask.loc[sel["df_idx"]].to_numpy()

    # Update parents/children among eligible rows only
    is_parent_row = sel["df_idx"].to_numpy() == sel["parent_df_idx"].to_numpy()
    eligible_parent = sel["eligible"].to_numpy() & is_parent_row
    eligible_child = sel["eligible"].to_numpy() & (~is_parent_row)

    parent_idxs = sel.loc[eligible_parent, "df_idx"].to_numpy()
    child_idxs  = sel.loc[eligible_child, "df_idx"].to_numpy()

    # Group id strategy:
    # - stable and readable: "version::<parent_record_id>"
    # This is perfect for later merging with other stages.
    #gid = "version::" + sel["parent_record_id"].astype("string")
    gid = sel["parent_record_id"].astype("string")

    # Apply
    if parent_idxs.size:
        df.loc[parent_idxs, hierarchy_col] = "parent"
        df.loc[parent_idxs, parent_id_col] = pd.NA
        df.loc[parent_idxs, group_id_col] = gid.loc[sel["df_idx"].isin(parent_idxs)].to_numpy()
    if child_idxs.size:
        df.loc[child_idxs, hierarchy_col] = f"parent - duplicate ({label_duplicate_suffix})"
        df.loc[child_idxs, parent_id_col] = sel.loc[sel["df_idx"].isin(child_idxs), "parent_record_id"].to_numpy()
        df.loc[child_idxs, group_id_col] = gid.loc[sel["df_idx"].isin(child_idxs)].to_numpy()

    metrics["n_rows_updated"] = int(parent_idxs.size + child_idxs.size)
    metrics["n_parents_set"] = int(parent_idxs.size)
    metrics["n_children_set"] = int(child_idxs.size)

    metrics["runtime_s"] = time.perf_counter() - t0
    return df, metrics


# ------------------------------------------------------------
# Example usage
# ------------------------------------------------------------
# df2, m = unify_groups_by_version_links(
#     data_out,
#     overwrite_mode="unlabeled_only",   # safest: only fill missing
#     choose_parent="oldest",
#     date_col="date_first_seen",
#     chunk_size=300_000,
# )
# print(m)


In [11]:
data_out, metric = unify_groups_by_version_links(
    data_clean,
    overwrite_mode="any",   
    choose_parent="oldest",
    date_col="date_first_seen",
    chunk_size=300_000,
)
print(metric)

{'n_rows_df': 8084860, 'n_rows_with_any_version_signal': 6460876, 'n_rows_considered_in_graph': 6460876, 'n_edges_unions_attempted': 178731, 'n_unions_done': 83495, 'n_components_total': 6377381, 'n_components_size_ge_2': 58022, 'n_rows_in_components_size_ge_2': 141517, 'n_rows_eligible_to_update': 8084860, 'n_rows_updated': 141517, 'n_parents_set': 58022, 'n_children_set': 83495, 'runtime_s': 14.53807227499783}


In [12]:
from pathlib import Path
import json
import pandas as pd

OUT_DIR = Path("outputs_new")
OUT_DIR.mkdir(parents=True, exist_ok=True)

# Save metrics
with open(OUT_DIR / "dedupe_metrics_version_links.json", "w") as f:
    json.dump(metric, f, indent=2)

#pd.DataFrame(metric).to_csv(OUT_DIR / "dedupe_metrics_version_links.csv", index=False)

# Save dataset
data_out.to_parquet(OUT_DIR / "dedupe_data_out_version_links.parquet", index=False)

print("✅ All files saved to outputs/")


✅ All files saved to outputs/


In [13]:
data_out['records_hierarchy'].value_counts()

records_hierarchy
parent                                           7439323
parent - duplicate (SSRN)                         173032
parent - duplicate (Zenodo)                        92230
parent - duplicate (version-link)                  83495
parent - duplicate (arXiv)                         61221
                                                  ...   
parent - duplicate (Bepress Legal Repository)          1
parent - duplicate (Encyclopedia)                      1
parent - duplicate (BioHackrXiv)                       1
parent - duplicate (Therapoid)                         1
parent - duplicate (searchRxiv)                        1
Name: count, Length: 106, dtype: int64

In [14]:
data_out

,record_id,server_name,backend,doi,doi_url,landing_page_url,title,authors_flat,institutions_flat,countries_flat,relations_json,version_label,is_version_of,is_preprint_of,has_preprint,has_review,has_published_version,published_version_ids_json,version_of_ids_json,update_to_json,raw_relationships_json,records_hierarchy,date_first_seen,publication_year_first_seen,records_hierarchy_backup,parent_record_id,dup_group_id,authors_fp_tokenbag,title_clean_v2,authors_fp_last_initial,authors_fp_last,records_hierarchy_dedupe_backup,parent_record_id_dedupe_backup,dup_group_id_dedupe_backup
0,crossref::10.21467/preprints.48,AIJR Preprints,crossref,10.21467/preprints.48,https://doi.org/10.21467/preprints.48,https://preprints.aijr.org/index.php/ap/prepri...,"Bird’s Eye View on the Diagnosis, Treatment, &...","Panchalingala, Sai Bhargavi",None,None,None,None,,,,,false,None,None,None,None,parent,2020-05-03,2020,parent,None,None,None,None,None,None,parent,None,None
1,crossref::10.21467/preprints.43,AIJR Preprints,crossref,10.21467/preprints.43,https://doi.org/10.21467/preprints.43,https://preprints.aijr.org/index.php/ap/prepri...,Doxycycline and Minocycline Drugs as a Treatme...,"Mostafa, Mohamed",None,None,None,None,,,,,false,None,None,None,None,parent,2020-04-25,2020,parent,None,None,mohamed_mostafa,doxycycline and minocycline drugs as a treatme...,mostafa|m,None,parent,None,None
2,crossref::10.21467/preprints.39,AIJR Preprints,crossref,10.21467/preprints.39,https://doi.org/10.21467/preprints.39,https://preprints.aijr.org/index.php/ap/prepri...,A Genetic Perspective of 2019-nCoV in Relation...,"Dasgupta, Rimjhim",None,None,None,None,,,,,false,None,None,None,None,parent,2020-04-16,2020,parent,None,None,dasgupta_rimjhim,a genetic perspective of 2019 ncov in relation...,dasgupta|r,None,parent,None,None
3,crossref::10.21467/preprints.38,AIJR Preprints,crossref,10.21467/preprints.38,https://doi.org/10.21467/preprints.38,https://preprints.aijr.org/index.php/ap/prepri...,Marine Algae as a Natural Source for Antiviral...,"Musale, Amar S; G., Raja Krishna Kumar; Sapre,...",None,None,None,None,,,,,false,None,None,None,None,parent,2020-04-15,2020,parent,None,None,None,None,None,None,parent,None,None
4,crossref::10.21467/preprints.36,AIJR Preprints,crossref,10.21467/preprints.36,https://doi.org/10.21467/preprints.36,https://preprints.aijr.org/index.php/ap/prepri...,Possible Prevention of COVID 19 by Using Linol...,"Subhash, Venkata; G, Raja Krishna Kumar; Sapre...",None,None,None,None,,,,,false,None,None,None,None,parent,2020-04-15,2020,parent,None,None,None,None,None,None,parent,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8410089,openalex::W999325625,viXra,openalex,None,None,https://vixra.org/pdf/1409.0090v1.pdf,Three Objections to Modern Physics,Lubomir Vlcek,None,None,None,None,None,None,None,None,None,None,None,None,None,parent,2014-09-01,2014,parent,None,None,lubomir_vlcek,three objections to modern physics,vlcek|l,None,parent,None,None
8410090,openalex::W999460032,viXra,openalex,None,None,https://vixra.org/abs/1112.0094,Particle Mass Ratios,DT Froedge,None,None,None,None,None,None,None,None,None,None,None,None,None,parent,2011-12-01,2011,parent,None,None,dt_froedge,particle mass ratios,froedge|d,None,parent,None,None
8410091,openalex::W99967155,viXra,openalex,None,None,https://vixra.org/pdf/1406.0019v1.pdf,Quantum FFF Theory Proposals for Some Unsolved...,Leo Vuyk,None,None,None,None,None,None,None,None,None,None,None,None,None,parent,2014-06-01,2014,parent,None,None,leo_vuyk,quantum fff theory proposals for some unsolved...,vuyk|l,None,parent,None,None
8410092,openalex::W999790414,viXra,openalex,None,None,https://vixra.org/pdf/1306.0105v3.pdf,Investigation of the Formalism of Particle Dyn...,Chi-Yi Chen,None,None,None,None,None,None,None,None,None,None,None,None,None,parent,2013-06-01,2013,parent,None,None,chen_chi_yi,investigation of the formal

In [15]:
# to check records_hierarchy

In [16]:
pattern = "10.26434/chemrxiv.13102877"


mask = data_out['doi'].str.contains(pattern, regex=False, na=False)
result = data_out[mask]
print(len(result))
result

32


,record_id,server_name,backend,doi,doi_url,landing_page_url,title,authors_flat,institutions_flat,countries_flat,relations_json,version_label,is_version_of,is_preprint_of,has_preprint,has_review,has_published_version,published_version_ids_json,version_of_ids_json,update_to_json,raw_relationships_json,records_hierarchy,date_first_seen,publication_year_first_seen,records_hierarchy_backup,parent_record_id,dup_group_id,authors_fp_tokenbag,title_clean_v2,authors_fp_last_initial,authors_fp_last,records_hierarchy_dedupe_backup,parent_record_id_dedupe_backup,dup_group_id_dedupe_backup
279942,crossref::10.26434/chemrxiv.13102877.v1,ChemRxiv,crossref,10.26434/chemrxiv.13102877.v1,https://doi.org/10.26434/chemrxiv.13102877.v1,https://chemrxiv.org/engage/chemrxiv/article-d...,Aptamers for Detection and Diagnostics (ADD): ...,"Datta, Shoumen",MIT,None,"{""is-version-of"": [{""asserted-by"": ""subject"", ...",None,10.26434/chemrxiv.13102877.v2;10.26434/chemrxi...,,,,false,None,None,None,None,parent,2020-10-23,2020,parent,<NA>,crossref::10.26434/chemrxiv.13102877.v1,datta_shoumen,aptamers for detection and diagnostics add pro...,datta|s,None,parent,None,16854196709595633026
279972,crossref::10.26434/chemrxiv.13102877.v2,ChemRxiv,crossref,10.26434/chemrxiv.13102877.v2,https://doi.org/10.26434/chemrxiv.13102877.v2,https://chemrxiv.org/engage/chemrxiv/article-d...,Aptamers for Detection and Diagnostics (ADD): ...,"Datta, Shoumen",MIT,None,"{""is-version-of"": [{""asserted-by"": ""subject"", ...",None,10.26434/chemrxiv.13102877.v1;10.26434/chemrxi...,,,,false,None,None,None,None,parent - duplicate (version-link),2020-10-27,2020,parent,crossref::10.26434/chemrxiv.13102877.v1,crossref::10.26434/chemrxiv.13102877.v1,datta_shoumen,aptamers for detection and diagnostics add pro...,None,None,parent - duplicate (ChemRxiv),crossref::10.26434/chemrxiv.13102877.v1,16854196709595633026
279999,crossref::10.26434/chemrxiv.13102877.v3,ChemRxiv,crossref,10.26434/chemrxiv.13102877.v3,https://doi.org/10.26434/chemrxiv.13102877.v3,https://chemrxiv.org/engage/chemrxiv/article-d...,Aptamers for Detection and Diagnostics (ADD) i...,"Datta, Shoumen",MIT,None,"{""is-version-of"": [{""asserted-by"": ""subject"", ...",None,10.26434/chemrxiv.13102877.v1;10.26434/chemrxi...,,,,false,None,None,None,None,parent - duplicate (version-link),2020-11-06,2020,parent,crossref::10.26434/chemrxiv.13102877.v1,crossref::10.26434/chemrxiv.13102877.v1,datta_shoumen,aptamers for detection and diagnostics add is ...,datta|s,None,parent,None,15378799223749550050
280030,crossref::10.26434/chemrxiv.13102877.v4,ChemRxiv,crossref,10.26434/chemrxiv.13102877.v4,https://doi.org/10.26434/chemrxiv.13102877.v4,https://chemrxiv.org/engage/chemrxiv/article-d...,Aptamers for Detection and Diagnostics (ADD) i...,"Datta, Shoumen",MIT,None,"{""is-version-of"": [{""asserted-by"": ""subject"", ...",None,10.26434/chemrxiv.13102877.v1;10.26434/chemrxi...,,,,false,None,None,None,None,parent - duplicate (version-link),2020-11-09,2020,parent,crossref::10.26434/chemrxiv.13102877.v1,crossref::10.26434/chemrxiv.13102877.v1,datta_shoumen,aptamers for detection and diagnostics add is ...,None,None,parent - duplicate (ChemRxiv),crossref::10.26434/chemrxiv.13102877.v3,15378799223749550050
280070,crossref::10.26434/chemrxiv.13102877.v5,ChemRxiv,crossref,10.26434/chemrxiv.13102877.v5,https://doi.org/10.26434/chemrxiv.13102877.v5,https://chemrxiv.org/engage/chemrxiv/article-d...,Aptamers for Detection and Diagnostics (ADD) i...,"Datta, Shoumen",MIT,None,"{""is-version-of"": [{""asserted-by"": ""subject"", ...",None,10.26434/chemrxiv.13102877.v1;10.26434/chemrxi...,,,,false,None,None,None,None,parent - duplicate (version-link),2020-11-11,2020,parent,crossref::10.26434/chemrxiv.13102877.v1,crossref::10.26434/chemrxiv.13102877.v1,datta_shoumen,aptamers for detection and diagnostics add is ...,None,None,parent - duplicate (ChemRxiv),crossref::10.26434/chemrxiv.13102877.v3,15378799223749550050
280100,crossref::10.26434/chemrxiv

In [17]:
result['dup_group_id'].value_counts()

dup_group_id
crossref::10.26434/chemrxiv.13102877.v1    32
Name: count, dtype: int64

In [18]:
result['parent_record_id'].value_counts()

parent_record_id
crossref::10.26434/chemrxiv.13102877.v1    31
Name: count, dtype: int64

In [20]:
result['records_hierarchy'].value_counts()

records_hierarchy
parent - duplicate (version-link)    31
parent                                1
Name: count, dtype: int64